In [2]:
pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torchvision

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
    --------------------------------------- 0.0/1.3 MB 330.3 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.3 MB 365.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.3 MB 554.9 kB/s eta 0:00:03
   ------ --------------------------------- 0.2/1.3 MB 737.3 kB/s eta 0:00:02
   -------- ------------------------------- 0.3/1.3 MB 896.4 kB/s eta 0:00:02
   ----------- ---------------------------- 0.4/1.3 MB 1.0 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.3 MB 1.0 MB/s eta 0:00:01
   --------------- ------------------------ 

In [5]:
pip install moviepy

  Using cached moviepy-1.0.3.tar.gz (388 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached decorator-4.4.2-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached imageio_ffmpeg-0.5.1-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached proglog-0.1.10-py3-none-any.whl.metadata (639 bytes)
Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
Using cached imageio_ffmpeg-0.5.1-py3-none-win_amd64.whl (22.6 MB)
Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110765 sha256=c8963e0f71963adc6bbb9eb2da20e378979bb9b4f6b72e24eaf92fd5ee2b208a
  Stored in directory: c:\users\aravi\appdata\local\pip\cache\wheels\df\ba\4b\0917fc0c8833c8ba7016565fc975b74c67bc8610806e930272
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled d

In [1]:
import cv2
import torch  # PyTorch for YOLO
import numpy as np
from moviepy.editor import VideoFileClip

# Load YOLO model (e.g., YOLOv5)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Use YOLOv5 small version (YOLOv5s)

def detect_objects(frame):
    # Use YOLO to detect objects in the frame
    results = model(frame)
    # Get bounding boxes, confidence scores, and classes
    boxes = results.xyxy[0].numpy()  # x1, y1, x2, y2, confidence, class
    return boxes

def process_frame(frame, target_width=1080, target_height=1920):
    # Detect objects in the current frame
    boxes = detect_objects(frame)

    if len(boxes) > 0:
        # Select the largest object (you can modify this logic based on your needs)
        largest_box = max(boxes, key=lambda box: (box[2] - box[0]) * (box[3] - box[1]))

        # Get coordinates of the largest object
        x1, y1, x2, y2, _, _ = largest_box
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Calculate center of the object
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Determine cropping area to ensure the object is in the center of the vertical video
        crop_x1 = max(0, center_x - target_width // 2)
        crop_y1 = max(0, center_y - target_height // 2)
        crop_x2 = min(frame.shape[1], crop_x1 + target_width)
        crop_y2 = min(frame.shape[0], crop_y1 + target_height)

        # Crop the frame to vertical format
        cropped_frame = frame[crop_y1:crop_y2, crop_x1:crop_x2]

        # Resize the cropped frame to target size
        resized_frame = cv2.resize(cropped_frame, (target_width, target_height))
        return resized_frame
    else:
        # If no object is detected, return resized original frame
        return cv2.resize(frame, (target_width, target_height))

def convert_video_to_vertical(input_video_path, output_video_path):
    # Load the input video using moviepy
    clip = VideoFileClip(input_video_path)
    
    # Define the process function for each frame
    def process_frame_wrapper(frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for OpenCV
        processed_frame = process_frame(frame_rgb)
        return cv2.cvtColor(processed_frame, cv2.COLOR_RGB2BGR)  # Convert back to BGR for moviepy
    
    # Apply the process function to each frame of the video
    new_clip = clip.fl_image(process_frame_wrapper)
    
    # Write the new vertical video to output file
    new_clip.write_videofile(output_video_path, codec="libx264", audio_codec="aac")

# Example usage
input_video = "input.mp4"
output_video = "output.mp4"
convert_video_to_vertical(input_video, output_video)

Using cache found in C:\Users\aravi/.cache\torch\hub\ultralytics_yolov5_master


requirements: Ultralytics requirement ['setuptools>=70.0.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.3/1.3 MB 1.4 MB/s eta 0:00:0002
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.5.1
    Uninstalling setuptools-69.5.1:
      Successfully uninstalled setuptools-69.5.1

requirements: AutoUpdate success  31.5s, installed 1 package: ['setuptools>=70.0.0']
requirements:  Restart runtime or rerun command for updates to take effect



YOLOv5  2024-9-14 Python-3.12.4 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
